## Day 4 - Exercise 2

We will build on this morning exercise on unit conversion. Your task is to re-write the solution using the Object Oriented Programming paradigm.

To do this, think about the different steps in our solution. Reformulate them as classes with their corresponding methods and properties. Then bring them all together to accomplish what you want!

I've copied a version of the solutions below so you can easily refer to them:

In [ ]:
# This is actually a code cell
i_am_using_colab = 0                # <-- Change this to 1 if you are using Colab
if i_am_using_colab:
    !wget https://raw.githubusercontent.com/nuitrcs/pythonBootcamp_4Day/main/conversionMeasures.csv

### Loading and cleaning data stage

```python
# First we load the file, and make a list where each element is a line from the file
file_name = 'conversionMeasures.csv'
with open(file_name, 'r') as f:
    raw_data = f.readlines()

# Then we create a new list, where each element is a smaller list:
conversion_data = []
for line in raw_data:
    conversion_data.append(line.rstrip("\n").split(","))
```

Let's start with this first step where we obtain the data and we clean the data. Most research pipelines have a **preprocessing** step. The above represents this preprocessing step. In your own research, it will be much more complicated, but having a class that handles all the preprocesing will be very valuable!

In [ ]:
class Preprocessor():
    pass

OK, what methods and attributes do we want?

Regarding methods:
* Open and read the file.
* Clean the file lines.

We also need a way for the user to indicate the file's name. I'll do this in a particular way, but you may design this differently.

In [ ]:
class Preprocessor():
    def __init__(self):
        self.raw_data = []
        self.data = []
        self.file_name = []

    def open_file(self):
        with open(self.file_name, "r") as f:
            self.raw_data = f.readlines()
    
    def clean_data(self):
        for line in self.raw_data:
            self.data.append(line.rstrip("\n").split(","))
    
    def preprocess(self, file_name):
        self.file_name = file_name
        self.open_file()
        self.clean_data()
        return self.data

OK, let's try our preprocessor!

In [ ]:
# Step 1: choose the file
file_name = 'conversionMeasures.csv'

In [ ]:
# Step 2: Create a preprocessor instance:
preprocessor = Preprocessor()

In [ ]:
# Step 3: Ask your preprocessor to load and clean your file!
preprocessor.preprocess(file_name)

Nice! Let's save this in a variable though. Here is everything summarized:

In [ ]:
file_name = 'conversionMeasures.csv'
preprocessor = Preprocessor()
conversion_table = preprocessor.preprocess(file_name)

In [ ]:
conversion_table[0:5]

Take a look at how much effort was required once the class was created: $3$ lines!!

### Converter

```python
def converter(v, unit_1, unit_2):
    # Step 1: Check if input types are valid:
    if type(v) != int or type(v) != float:
        try:
            v = float(v)
        except TypeError:
            print(f'The value you are trying to convert must be an integer or a float, not a {type(v).__name__}')
            return
    if type(unit_1) != str or type(unit_2) != str:
        print("Your units must be strings. Get it together plz.")
        return
        
    # Step 2: Join strings that are separated
    unit_1 = "_".join(unit_1.lower().split(" ")) 
    unit_2 = "_".join(unit_2.lower().split(" "))
    
    # Step 3: Run the for loop we had before, to find the unit conversion
    for i in conversion_data:
        try:
            c = float(i[1])
        except ValueError:
            c1, c2 = i[1].split("/")
            c = float(c1) / float(c2)
        if i[0].lower() == unit_1 and i[2].lower() == unit_2:
            new_value = v * c
            print(f'I got it! Your value of {v} {unit_1} is the same as {new_value} {unit_2}')
            return
        elif i[2].lower() == unit_1 and i[0].lower() == unit_2:
            new_value = v / c
            print(f'I got it! Your value of {v} {unit_1} is the same as {new_value} {unit_2}')
            return
            
    # Step 4: No conversion found
    print("I don't know this conversion. Sooooooorry!")
    return
````

Converter is quite long. When things become very complex, usually that means we can create several classes out of them. In this case we'll keep it just one converter class:

In [ ]:
class UnitConverter():

    def __init__(self, conversion_table):
        self.conversion_table = conversion_table

    def convert_input(self, v):
        # Returns a float version of v if possible. Else it throws an error.
        if type(v) != int or type(v) != float:
            try:
                v = float(v)
            except TypeError:
                print(f'The value you are trying to convert must be an integer or a float, not a {type(v).__name__}')
                return
        return v
    
    def check_units(self, unit1, unit2):
        if type(unit1) != str or type(unit2) != str:
            print("Your units must be strings. Get it together plz.")
            return
        return
    
    def clean_units(self, unit_1, unit_2):
        unit_1 = "_".join(unit_1.lower().split(" "))
        unit_2 = "_".join(unit_2.lower().split(" "))
        return unit_1, unit_2
    
    def clean_conversion_factor(self, conv_fac):
        try:
            c = float(conv_fac)
        except ValueError:
            c1, c2 = conv_fac.split("/")
            c = float(c1) / float(c2)
        return c
    
    def convert_from_table(self, value, unit_1, unit_2):
        for i in self.conversion_table:
            cf = self.clean_conversion_factor(i[1])
            if i[0].lower() == unit_1 and i[2].lower() == unit_2:
                new_value = value * cf
                print(f'I got it! Your value of {value} {unit_1} is the same as {new_value} {unit_2}')
                return
            elif i[2].lower() == unit_1 and i[0].lower() == unit_2:
                new_value = value / cf
                print(f'I got it! Your value of {value} {unit_1} is the same as {new_value} {unit_2}')
                return
        print("Sorry. No conversion found.")
        return
    
    def convert(self, value, unit_1, unit_2):
        value = self.convert_input(value)
        self.check_units(unit_1, unit_2)
        unit_1, unit_2 = self.clean_units(unit_1, unit_2)
        self.convert_from_table(value, unit_1, unit_2)
        return

In [ ]:
converter = UnitConverter(conversion_table)
converter.convert(3, "kilometer", "meter")

### Testing

```python
# Step 1. Put the test cases in a list of dictionaries:
test_cases = [{"test_unit":   "pint",        "test_value": 2.5,   "final_unit": "mL"},
                {"test_unit": "cubic foot",  "test_value": 30,    "final_unit": "liter"},
                {"test_unit": "slug",        "test_value": "4.8", "final_unit": "pound"},
                {"test_unit": "slug",        "test_value": 27.0,  "final_unit": "snail"},
                {"test_unit": [],            "test_value": 2.5,   "final_unit": "meter"},
                {"test_unit": "kilometer",   "test_value": [],    "final_unit": "meter"},
                {"test_unit": "KM/H",        "test_value": 8.4,   "final_unit": "m/Sec"},
                {"test_unit": "ergs",        "test_value": 8.4,   "final_unit": "joule"},
                {"test_unit": "tablespoons", "test_value": 2,     "final_unit": "cup"}
]
````

```python
# Step 2. Create a function that checks our converter for all tests
def test_checker(f_converter, test_cases):
    n_case = 0
    for test in test_cases:
        try:
            f_converter(test["test_value"], test["test_unit"], test["final_unit"])
            print(f"(Test case number {n_case} passed)\n")
        except (TypeError, ValueError):
            print(f"(Test case number {n_case} failed)\n")
        n_case = n_case + 1
    return
````

I will add the test cases as a property of our test checker class (which I will name `Tester`).

Note that before we used to pass a **function** to the converter. Now let's pass an object of class `UnitConverter`.

In [ ]:
class ConversionTester():
    # Class property
    test_cases = [{"test_unit":   "pint",        "test_value": 2.5,   "final_unit": "mL"},
                {"test_unit": "cubic foot",  "test_value": 30,    "final_unit": "liter"},
                {"test_unit": "slug",        "test_value": "4.8", "final_unit": "pound"},
                {"test_unit": "slug",        "test_value": 27.0,  "final_unit": "snail"},
                {"test_unit": [],            "test_value": 2.5,   "final_unit": "meter"},
                {"test_unit": "kilometer",   "test_value": [],    "final_unit": "meter"},
                {"test_unit": "KM/H",        "test_value": 8.4,   "final_unit": "m/Sec"},
                {"test_unit": "ergs",        "test_value": 8.4,   "final_unit": "joule"},
                {"test_unit": "tablespoons", "test_value": 2,     "final_unit": "cup"}
    ]

    def test_converter(self, converter):
        # Note converter is a function (for now)!
        for n_case, test in enumerate(self.test_cases):
            try:
                converter.convert(test["test_value"], test["test_unit"], test["final_unit"])
                print(f"(Test case number {n_case} passed)\n")
            except:
                print(f"(Test case number {n_case} failed)\n")
        return

Notice I'm not defining an `__init__()` function. That's because, for now at least, I don't need it to do anything special for initialization.

In [ ]:
tester = ConversionTester()
tester.test_converter(converter)

### User Input

```python
def chat_bot_converter(f_converter):
    max_tries = 3
    tries = 0
    valid = 0
    while tries < max_tries:
        tries = tries + 1
        v = input("Please provide the value you want to convert -->  ")
        if type(v) != int or type(v) != float:
            try:
                v = float(v)
                valid = 1
                break
            except (TypeError, ValueError):
                if tries < max_tries:
                    print("Not a valid value. Your value must be an integer or a float, try again!")
                else:
                    print("I'm giving up")
                
    if valid:
        u1 = input("Please provide the original unit -->")
        u2 = input("Please provide the unit you want to convert to -->")
        f_converter(v, u1, u2)
    return
````

In [ ]:
class BotConverter():
    max_tries = 3
    def __init__(self):
        self.tries = 0
        self.valid = 0
        self.converter = None
    
    def add_converter(self, converter):
        self.converter = converter
        return
    
    def check_value(self, v):
        self.tries += 1
        if type(v) != int or type(v) != float:
            try:
                v = float(v)
                self.valid = 1
            except (TypeError, ValueError):
                if self.tries < self.max_tries:
                    print("Not a valid value. Your value must be an integer or a float, try again!")
                else:
                    print("I'm giving up")
        return v
    
    def input_loop(self):
        while (self.valid == 0) and (self.tries < self.max_tries):
            v = input("Please provide the value you want to convert -->  ")
            v = self.check_value(v)
        return v
    
    def chat(self):
        self.valid = 0
        self.tries = 0
        v = self.input_loop()
        if self.valid:
            u1 = input("Please provide the original unit -->  ")
            u2 = input("Please provide the unit you want to convert to -->  ")
            self.converter.convert(v, u1, u2)
        return


In [ ]:
bender = BotConverter()
bender.add_converter(converter)

In [ ]:
bender.chat()

### Conclusion

Great, we've OOPified everything now. In our case it happened in a very straighforward way, we mostly just turned little pieces of code into class methods. It helped that we had already organized it in stages before. BUT, this will not always be the case. The more experience you gain, the easier this will become, and the more powerful the systems you build will be!

Now that we have built the classes we want, the whole pipeline is very straighforward. Note that we could re-use this classes for further applications and projects, we could combine them in different order, etc.

Here's how things would look once all classes have been defined:

In [ ]:
# Preprocessing Stage
file_name = 'conversionMeasures.csv'
preprocessor = Preprocessor()
conversion_table = preprocessor.preprocess(file_name)

# Create Converter
converter = UnitConverter(conversion_table)

# Test Converter
tester = ConversionTester()
tester.test_converter(converter)

# Chat with the converter
bender = BotConverter()
bender.add_converter(converter)
bender.chat()